In [ ]:
# === IMPORTS AND SETUP ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def format_seconds(secs):
    mins = int(secs // 60)
    secs = int(secs % 60)
    return f"{mins} min {secs} sec"

In [ ]:
    import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def format_seconds(secs):
    mins = int(secs // 60)
    secs = int(secs % 60)
    return f"{mins} min {secs} sec"

In [ ]:
df = pd.read_csv(r'C:\Users\jakers\Downloads\trades_export.csv')
df['EnteredAt'] = pd.to_datetime(df['EnteredAt'])
df['ExitedAt'] = pd.to_datetime(df['ExitedAt'])
df['TradeDay'] = pd.to_datetime(df['TradeDay'])
df['Duration_sec'] = pd.to_timedelta(df['TradeDuration']).dt.total_seconds()
df['Was_Win'] = df['PnL'] > 0
df['Day'] = df['EnteredAt'].dt.day_name()
df['Month'] = df['EnteredAt'].dt.strftime('%B')
df['Direction'] = df['Type'].str.lower().str.strip()

In [1]:
# Load your real trading data
df = pd.read_csv(r'C:\Users\jakers\Downloads\trades_export.csv')
df['Was_Win'] = df['PnL'] > 0
df['Cumulative_PnL'] = df['PnL'].cumsum()
your_stats = {
    'win_rate': df['Was_Win'].mean(),
    'avg_win': df[df['Was_Win']]['PnL'].mean(),
    'avg_loss': abs(df[~df['Was_Win']]['PnL'].mean()),
    'n_trades': len(df),
    'trade_outcomes': df['PnL'].values
}

NameError: name 'pd' is not defined

In [ ]:
# Example system stats
systems = {
    'Poor':    {'win_rate': 0.3, 'avg_win': 30,  'avg_loss': 60},
    'Average': {'win_rate': 0.5, 'avg_win': 50,  'avg_loss': 50},
    'Good':    {'win_rate': 0.6, 'avg_win': 80,  'avg_loss': 40}
}
n_trades = 100  # Number of trades to simulate for each system

In [ ]:
def monte_carlo_sim(win_rate, avg_win, avg_loss, n_trades, n_simulations=100):
    sim_curves = []
    for _ in range(n_simulations):
        outcomes = np.random.choice(
            [avg_win, -avg_loss],
            size=n_trades,
            p=[win_rate, 1-win_rate]
        )
        sim_curves.append(np.cumsum(outcomes))
    return np.array(sim_curves)

In [ ]:
sim_results = {}
for name, stats in systems.items():
    sim_results[name] = monte_carlo_sim(
        stats['win_rate'], stats['avg_win'], stats['avg_loss'], n_trades
    )

# For your actual stats, use your real trade distribution for realism
your_sim = []
for _ in range(100):
    outcomes = np.random.choice(your_stats['trade_outcomes'], size=n_trades, replace=True)
    your_sim.append(np.cumsum(outcomes))
your_sim = np.array(your_sim)

In [ ]:
# === FULL DASHBOARD SUMMARY ===

total_pnl = df['PnL'].sum()
win_pct = (df['Was_Win'].mean() * 100)
avg_win = df[df['Was_Win']]['PnL'].mean()
avg_loss = df[~df['Was_Win']]['PnL'].mean()
win_loss_ratio = abs(avg_win / avg_loss) if avg_loss != 0 else np.nan
day_win_pct = (df.groupby('TradeDay')['PnL'].sum() > 0).mean() * 100
profit_factor = df[df['PnL'] > 0]['PnL'].sum() / abs(df[df['PnL'] < 0]['PnL'].sum())
num_days = df['TradeDay'].nunique()
most_active_day = df['Day'].value_counts().idxmax()
active_trades = df['Day'].value_counts().max()
most_profitable_day = df.groupby('Day')['PnL'].sum().idxmax()
least_profitable_day = df.groupby('Day')['PnL'].sum().idxmin()
most_profitable_day_pnl = df.groupby('Day')['PnL'].sum().max()
least_profitable_day_pnl = df.groupby('Day')['PnL'].sum().min()
total_lots = df['Size'].sum()
avg_trade_dur = format_seconds(df['Duration_sec'].mean())
avg_win_dur = format_seconds(df[df['Was_Win']]['Duration_sec'].mean())
avg_loss_dur = format_seconds(df[~df['Was_Win']]['Duration_sec'].mean())
best_trade = df.loc[df['PnL'].idxmax()]
worst_trade = df.loc[df['PnL'].idxmin()]
direction_pct = (df['Direction'] == 'short').mean() * 100 if 'short' in df['Direction'].unique() else np.nan
monthly_pnl = df.groupby('Month')['PnL'].sum()
daily_pnl = df.groupby('TradeDay')['PnL'].sum()
cumulative_daily = daily_pnl.cumsum()
best_day_pct = (daily_pnl.max() / total_pnl * 100) if total_pnl != 0 else 0

print("=== FULL DASHBOARD SUMMARY ===")
print(f"Total P&L: ${total_pnl:,.2f}")
print(f"Trade Win %: {win_pct:.2f}%")
print(f"Avg Win / Avg Loss: {win_loss_ratio:.2f}")
print(f"Avg Winning Trade: ${avg_win:.2f}")
print(f"Avg Losing Trade: ${avg_loss:.2f}")
print(f"Day Win %: {day_win_pct:.2f}%")
print(f"Profit Factor: {profit_factor:.2f}")
print(f"Best Day % of Total Profit: {best_day_pct:.2f}%")
print(f"Total Number of Trades: {len(df)}")
print(f"Total Number of Lots Traded: {total_lots}")
print(f"Average Trade Duration: {avg_trade_dur}")
print(f"Average Win Duration: {avg_win_dur}")
print(f"Average Loss Duration: {avg_loss_dur}")
print(f"Most Active Day: {most_active_day} ({active_trades} trades)")
print(f"Most Profitable Day: {most_profitable_day} (${most_profitable_day_pnl:.2f})")
print(f"Least Profitable Day: {least_profitable_day} (${least_profitable_day_pnl:.2f})")
print(f"Best Trade: ${best_trade['PnL']:.2f} | {best_trade['Type']} {best_trade['Size']} {best_trade['ContractName']} @ {best_trade['EntryPrice']} Exited @ {best_trade['ExitPrice']} {best_trade['ExitedAt']}")
print(f"Worst Trade: ${worst_trade['PnL']:.2f} | {worst_trade['Type']} {worst_trade['Size']} {worst_trade['ContractName']} @ {worst_trade['EntryPrice']} Exited @ {worst_trade['ExitPrice']} {worst_trade['ExitedAt']}")
print(f"Trade Direction % (Short): {direction_pct:.2f}%")
print(f"Monthly P/L: {monthly_pnl.to_dict()}")
print(f"\nYou've made {len(df)} trades over {num_days} days. Consider focusing on quality over quantity to avoid overtrading.")

In [ ]:
import matplotlib.dates as mdates

# --- 1. Daily Net Cumulative P&L ---
plt.figure(figsize=(12,6))
plt.plot(cumulative_daily.index, cumulative_daily.values, marker='o', linewidth=2, color='navy')
plt.title('Daily Net Cumulative P&L', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative P&L ($)', fontsize=12)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- 2. Net Daily P&L (Bar Chart with Color Coding) ---
plt.figure(figsize=(12,6))
colors = ['green' if v >= 0 else 'red' for v in daily_pnl.values]
plt.bar(daily_pnl.index, daily_pnl.values, color=colors, edgecolor='black')
plt.title('Net Daily P&L', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Net P&L ($)', fontsize=12)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.axhline(0, color='black', linewidth=1)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- 3. Trade Duration Analysis (Histogram with Stats) ---
plt.figure(figsize=(12,5))
plt.hist(df['Duration_sec']/60, bins=20, color='skyblue', edgecolor='black', alpha=0.8)
plt.title('Trade Duration Distribution (minutes)', fontsize=16)
plt.xlabel('Duration (minutes)', fontsize=12)
plt.ylabel('Number of Trades', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.axvline(df['Duration_sec'].mean()/60, color='orange', linestyle='dashed', linewidth=2, label=f"Avg: {format_seconds(df['Duration_sec'].mean())}")
plt.legend()
plt.tight_layout()
plt.show()

# --- 4. P&L Distribution (Histogram) ---
plt.figure(figsize=(12,5))
plt.hist(df['PnL'], bins=30, color='purple', edgecolor='black', alpha=0.7)
plt.title('P&L per Trade Distribution', fontsize=16)
plt.xlabel('P&L per Trade ($)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.axvline(df['PnL'].mean(), color='orange', linestyle='dashed', linewidth=2, label=f"Avg: ${df['PnL'].mean():.2f}")
plt.legend()
plt.tight_layout()
plt.show()

# --- 5. Trades by Day of Week ---
plt.figure(figsize=(10,5))
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
trade_counts = df['Day'].value_counts().reindex(day_order).dropna()
plt.bar(trade_counts.index, trade_counts.values, color='teal', edgecolor='black')
plt.title('Number of Trades by Day of Week', fontsize=16)
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Number of Trades', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6, axis='y')
plt.tight_layout()
plt.show()

# --- 6. Win Rate by Day of Week ---
plt.figure(figsize=(10,5))
win_rate_by_day = df.groupby('Day')['Was_Win'].mean().reindex(day_order).dropna() * 100
plt.bar(win_rate_by_day.index, win_rate_by_day.values, color='goldenrod', edgecolor='black')
plt.title('Win Rate by Day of Week', fontsize=16)
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Win Rate (%)', fontsize=12)
plt.ylim(0, 100)
plt.grid(True, linestyle='--', alpha=0.6, axis='y')
plt.tight_layout()
plt.show()